In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_rows', 1000)

In [9]:
df = pd.read_csv('./../output/slo_montly_data.csv', sep=';')
df.head()

,Group,Journey,AvailabilitySLA,AvailabilitySlo,ExperienceSlo,LatencySLA,LatencySlo,Feature,Source,year,month,month_name,total,ava,ava_prop,exp,exp_prop,lat
0,Sales,Checkout Order,0.95,0.99,0.99,1000,1.0,Login,LoginController:PreLogin,2020.0,2020-01,Jan,399598.0,395000.0,0.988456,391156.0,0.978908,0.988277
1,Sales,Checkout Order,0.95,0.99,0.99,1000,1.0,Login,LoginController:PreLogin,2020.0,2020-02,Feb,392950.0,388520.0,0.988811,384788.0,0.979183,1.020870
2,Sales,Checkout Order,0.95,0.99,0.99,1000,1.0,Login,LoginController:PreLogin,2020.0,2020-03,Mar,418244.0,413587.0,0.988837,409668.0,0.979533,0.988957
3,Sales,Checkout Order,0.95,0.99,0.99,1000,1.0,Login,LoginController:PreLogin,2020.0,2020-04,Apr,391036.0,386515.0,0.988369,382628.0,0.978512,0.998299
4,Sales,Checkout Order,0.95,0.99,0.99,1000,1.0,Login,LoginController:PreLogin,2020.0,2020-05,May,393065.0,388452.0,0.988266,385088.0,0.979741,1.000371


In [32]:
columns = ['month','Feature', 'AvailabilitySLA', 'AvailabilitySlo', 'ExperienceSlo', 'LatencySLA', 'LatencySlo']
grouped = df.groupby(columns).aggregate({'Source':'count', 'ava_prop': 'min', 'exp_prop': 'min', 'lat': 'max' }).reset_index()

grouped['ava_debt'] = grouped['AvailabilitySlo'] - grouped['ava_prop']
grouped.loc[grouped['ava_debt'] < 0, 'ava_debt'] = 0

grouped['exp_debt'] = grouped['ExperienceSlo'] - grouped['exp_prop']
grouped.loc[grouped['exp_debt'] < 0, 'exp_debt'] = 0


grouped['lat_debt'] = grouped['LatencySlo'] - grouped['lat']
grouped.loc[grouped['lat_debt'] > 0, 'lat_debt'] = 0
grouped['lat_debt'] = grouped['lat_debt'] * -1


def debt_per(x, target):
    tmp = grouped[grouped['month'] == x['month']][target].sum()
    return x[target] / tmp if tmp > 0 else 0
def debt_ava_per(x):
    return debt_per(x, 'ava_debt')
def debt_exp_per(x):    
    return debt_per(x, 'exp_debt')
def debt_lat_per(x):    
    return debt_per(x, 'lat_debt')


grouped['ava_debt_per'] = grouped.apply(debt_ava_per, axis=1) 
grouped['exp_debt_per'] = grouped.apply(debt_exp_per, axis=1) 
grouped['lat_debt_per'] = grouped.apply(debt_lat_per, axis=1)



In [33]:
fig = px.line(grouped, x="month", y="ava_debt", color="Feature", title="Availability Feature Debt Value Per Month")
fig.show()


In [35]:
fig = px.bar(grouped, x="month", y="ava_debt_per", color="Feature", title="Availability Feature Debt Percentage Per Month")
fig.show()

In [36]:
fig = px.line(grouped, x="month", y="exp_debt", color="Feature", title="Experience Feature Debt Value Per Month")
fig.show()

In [37]:
fig = px.bar(grouped, x="month", y="exp_debt_per", color="Feature", title="Experience Feature Debt Percentage Per Month")
fig.show()

In [38]:
fig = px.line(grouped, x="month", y="lat_debt", color="Feature", title="Latency Feature Debt Value Per Month")
fig.show()

In [39]:
fig = px.bar(grouped, x="month", y="lat_debt_per", color="Feature", title="Latency Feature Debt Percentage Per Month")
fig.show()